# Tutorial - Drawing Interactive Diagrams with Dual Canvases 

This tutorial explains how to use the `dual_canvas_helper` jQuery plugin to implement
diagrams that respond to mouse events.  The tutorial explains how to create the diagrams
as "standalone pure Javascript" which can be embedded in any HTML page and also shows
how to implement the diagrams inside a Jupyter IPython notebook.

The tutorial is intended for programmers who have some knowledge of Javascript, Python
Jupyter, and HTML 5 libraries such as jQuery.  The tutorial also makes use of
[proxy widget features](https://github.com/AaronWatters/jp_proxy_widget) without
explaining them in great detail -- please see the proxy widget tutorial for additional
discussion.

The dual canvas implementation is a `jQueryUI` plugin which is built using `jQuery`
and some other plugin functionality.  It requires several Javascript libraries
to be loaded.

- `jQuery`,
- `jQueryUI`,
- `js/canvas_2d_widget_helper.js` -- the single canvas `jQuery` plugin, and
- `js/dual_canvas_helper.js` -- the dual canvas `jQuery` plugin.

The Python wrapper `dual_canvas.py` makes the dual canvas conveniently available
as IPython widgets, and it provides a convenience function for loading the
required javascript libraries: `dual_canvas.load_requirements()`.

In [ ]:
from jp_doodle import dual_canvas
dual_canvas.load_requirements()

# Drawing objects

The `dual_canvas_helper` Plugin attaches a factory function to
jQuery elements `element.dual_canvas_helper(parent, config)` which creates
an HTML canvas area associated with the element and attached as a child
of the parent.  Methods attached to the element like `element.rect({...})`
draw shapes onto the drawing area.

Below we attach a canvas to a `jp_proxy_widget` IPython widget and
draw a number of objects onto the canvas using Javascript.

In [ ]:
from IPython.display import HTML, display
import jp_proxy_widget

# Create a proxy widget to contain the canvas.
canvas0 = jp_proxy_widget.JSProxyWidget()

# Load javascript for the widget.
canvas0.js_init("""
// Empty the js_proxy jQuery element associated with the widget.
element.empty();

// Attach an information div to the element to display event feedback.
var info_area = $("<div>Dual canvas feedback will show here.</div>").appendTo(element);

// Attach a dual canvas associated with the element as a child of the element
// configured with width 400 and height 200.
var config = {
            width: 400,
            height: 200,
        };
element.dual_canvas_helper(element, config);

// Draw some named elements on the canvas.
// A filled yellow circle (disk) named "Colonel Mustard
element.circle({name: "Colonel Mustard", x:100, y:150, r:90, color:"yellow"});
// A filled red rectangle named "Miss Scarlett"
element.rect({name: "Miss Scarlett", x:100, y:130, w:100, h:20, color: "red"});
// An unfilled white circle named "Mrs. White"
element.circle({
    name: "Mrs. White", x:100, y:150, r:58, fill:false, 
    color:"white", lineWidth: 14});
// An unfilled blue rectangle named Mrs. Peacock
element.rect({
    name: "Mrs. Peacock", x:40, y:110, w:100, h:20,
    color: "blue", lineWidth: 10, degrees:70, fill:false});
element.line({
    name: "Professor Plum", x1:190, y1:100, x2:10, y2:200,
    color:"purple", lineWidth: 20})
// A brown filled polygon (triangle) named Micky
element.polygon({
    name: "Micky",
    points: [[210, 10], [210, 110], [290, 60]],
    color: "brown",
})
// A green polyline named Mr. Green
element.polygon({
    name: "Mr. Green", fill:false, close:false, color: "green",
    lineWidth: 14, points: [[210, 10], [210, 110], [290, 60]]
})
// A magenta text named Pluto
element.text({
    name: "Pluto", text: "The Republic", font: "20px Arial",
    x: 20, y:20, degrees: 5, color:"magenta"
})

// Center and scale the figure to fit in the available area.
element.fit()

// Attach a mouse move event which indicates what object the mouse is over.
var on_mouse_move = function(event) {
    if (event.canvas_name) {
        info_area.html("<div>You are over the object named " + event.canvas_name + "</div>");
    } else {
        info_area.html("<div>You are not over anybody.</div>");
    }
};
element.on_canvas_event("mousemove", on_mouse_move);

$("<div>Please mouse over the canvas area to see event feedback.</div>").appendTo(element)
""")

# Show the canvas
display(canvas0)

# Redraws, animation, and the object list

The `dual canvas` implementation maintains a sequence of
drawn objects in order to support redrawing the canvas.
Every redraw clears the canvas and then iterates through the object list
to draw each of the objects again in the order they were added to the
canvas.

Furthermore named objects may be modified or deleted in order
to change the canvas before a redraw.  By modifying and
redrawing a canvas a program can implement animations
and other interactive features.  Unnamed objects (objects
created with no name specified) cannot be modified.  Changes
to objects or object deletions automatically trigger a canvas
redraw.

Below we draw a clock face with numbers and add a seconds hand.
We animate rotation for the seconds hand by updating the `degrees` rotation
attribute for each animation frame based on the current time.

We also add or delete a red circle for a blinking dot effect
every other second.  The blinking red circle sits atop a fixed
yellow circle.

In [ ]:

# Create a proxy widget to contain the canvas.
canvas1 = jp_proxy_widget.JSProxyWidget()

# Load javascript for the widget.
canvas1.js_init("""
// Empty the js_proxy jQuery element associated with the widget.
element.empty();

// Attach a dual canvas associated with the element as a child of the element
// configured with width 400 and height 200.
var config = {
            width: 400,
            height: 400,
        };
element.dual_canvas_helper(element, config);

// Draw a clock face.
five_seconds = Math.PI / 6.0;
twelve_oclock = Math.PI / 2.0;
outer_radius = 100;
inner_radius = 80;

// Draw the face background circle.
element.circle({x:0, y:0, r:100, color:"cyan"});

// Draw the numbers on the clock.
for (var i=1; i<=12; i++) {
    var angle = twelve_oclock - i * five_seconds;
    element.text({
        text: ""+i, font: "20px Arial", color: "magenta", align: "center",
        x: Math.cos(angle)*inner_radius, y: Math.sin(angle)*inner_radius - 7})
}
// Draw a "seconds hand" and name it so we can change it later.
element.rect({
    name: "seconds hand",
    x:0, y:0, w:inner_radius-15, h:1,
    color: "blue", lineWidth: 10, degrees:90, fill:false});
    
// Note that there is no blinking dot yet.
var dot_visible = false;
element.circle({name: "background dot", x:100, y:100, r:12, color:"yellow"});

// Every animation frame, adjust the seconds hand using the time.
// Also add or delete a blinking dot every other second.
var animate = function () {
    var seconds = ((new Date()).getTime() * 0.001) % 60;
    var degrees = - 6 * seconds;
    // Adjust the seconds hand.
    element.change_element("seconds hand", {degrees: degrees});
    // every other second create or delete the blinking dot
    if ((seconds % 2) < 1) {
        if (!dot_visible) {
            // Add the blinking dot.
            element.circle({name: "blinking dot", x:100, y:100, r:10, color:"red"});
            dot_visible = true;
        }
    } else if (dot_visible) {
        // Remove the blinking dot.
        element.forget_objects(["blinking dot"]);
        dot_visible = false;
    }
    // Repeat the animation again on the next animation iteration.
    requestAnimationFrame(animate);
}
// Adjust the canvas coordinate transform to center the drawn objects on the canvas.
element.fit()
// Start the animation
animate();
""")

display(canvas1)

In [ ]:
# http://population.us/states
STATES = """
Alabama	4858979	92.7	0.329
Alaska	738432	1.1	0.782
Arizona	6828065	59.9	1.329
Arkansas	2978204	56	0.424
California	39144818	239.1	0.995
Colorado	5456574	52.4	1.645
Connecticut	3590886	647.8	0.094
Delaware	945934	380.1	1.047
District of Columbia	672228	9836.5	2.241
Florida	20271272	308.3	1.517
Georgia	10214860	171.9	1.065
Hawaii	1431603	131	1.027
Idaho	1654930	19.8	1.09
Illinois	12859995	222.1	0.046
Indiana	6619680	181.8	0.416
Iowa	3123899	55.5	0.504
Kansas	2911641	35.4	0.407
Kentucky	4425092	109.5	0.392
Louisiana	4670724	89.2	0.599
Maine	1329328	37.6	0.015
Maryland	6006401	484.2	0.794
Massachusetts	6794422	643.8	0.743
Michigan	9922576	102.6	0.079
Minnesota	5489594	63.1	0.691
Mississippi	2992333	61.8	0.168
Missouri	6083672	87.3	0.314
Montana	1032949	7	0.865
Nebraska	1896190	24.5	0.753
Nevada	2890845	26.1	1.371
New Hampshire	1330608	142.3	0.214
New Jersey	8958013	1027	0.375
New Mexico	2085109	17.1	0.251
New York	19795791	362.9	0.427
North Carolina	10042802	186.6	1.042
North Dakota	756927	10.7	2.391
Ohio	11613423	259.1	0.133
Oklahoma	3911338	56	0.839
Oregon	4028977	41	1.012
Pennsylvania	12802503	278	0.157
Rhode Island	1056298	683.7	0.071
South Carolina	4896146	152.9	1.144
South Dakota	858469	11.1	1.065
Tennessee	6600299	156.6	0.789
Texas	27469114	102.3	1.783
Utah	2995919	35.3	1.625
Vermont	626042	65.1	0.01
Virginia	8382993	196	0.937
Washington	7170351	100.6	1.292
West Virginia	1844128	76.1	-0.096
Wisconsin	5771337	88.1	0.295
Wyoming	586107	6	0.785
"""